<a href="https://colab.research.google.com/github/nuot/AL-Clean/blob/main/AssistantListing_ObliClean_InProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assistant Listing Obligation data cleaning project In Progress** 
<br>

Author: Nuo Tian

**Last Updated: 02/16/2021**

In [6]:
import pandas as pd
import re
data = pd.read_csv('AssistanceListings.csv')

In [3]:
obligations = data['Obligations (122)']

In [4]:
obligations

0       (Project Grants) FY 19$4,704,045.00; FY 20 est...
1       (Salaries and Expenses) FY 19$288,228,684.00; ...
2       (Salaries and Expenses) FY 19$8,832,031.00; FY...
3       (Direct Payments with Unrestricted Use) FY 19$...
4       (Direct Loans) FY 19$3,205,858,000.00; FY 20 e...
                              ...                        
2270    (Project Grants) FY 19 FY 20 FY 21 FY 18 FY 17...
2271    (Project Grants) FY 19 FY 20 FY 21 FY 18 FY 16...
2272    (Project Grants) FY 19 FY 20 FY 21 FY 18 FY 17...
2273    (Project Grants) FY 19$17,095,000.00; FY 20 es...
2274    (Project Grants) FY 19$30,000,000.00; FY 20 es...
Name: Obligations (122), Length: 2275, dtype: object

In [9]:
types = []
for row in obligations:
  a = re.findall(r'\((.*?)\) FY',str(row))
  if a:
    types.append(a)
  else:
    types.append('None')

types

[['Project Grants'],
 ['Salaries and Expenses'],
 ['Salaries and Expenses'],
 ['Direct Payments with Unrestricted Use'],
 ['Direct Loans', 'Direct Payments with Unrestricted Use'],
 ['Direct Payments with Unrestricted Use'],
 ['Direct Payments with Unrestricted Use'],
 ['Direct Payments with Unrestricted Use',
  'Direct Payments with Unrestricted Use'],
 ['Direct Loans'],
 ['Direct Payments with Unrestricted Use'],
 ['Cooperative Agreements', 'Salaries and Expenses'],
 ['Direct Payments for Specified Use'],
 ['Direct Payments for Specified Use'],
 ['Direct Payments for Specified Use'],
 ['Project Grants', 'Salaries and Expenses'],
 ['Direct Payments with Unrestricted Use'],
 ['Guaranteed/Insured Loans'],
 ['Direct Payments with Unrestricted Use'],
 ['Direct Payments with Unrestricted Use'],
 ['Direct Payments with Unrestricted Use'],
 ['Direct Payments for Specified Use'],
 ['Direct Payments for Specified Use'],
 ['Direct Payments for Specified Use'],
 ['Direct Payments with Unrestrict

In [8]:
data['Obligation Types'] = types

In [13]:
data

,Program Title,Program Number,Popular Name (020),Federal Agency (030),Authorization (040),Objectives (050),Types of Assistance (060),Uses and Use Restrictions (070),Applicant Eligibility (081),Beneficiary Eligibility (082),Credentials/Documentation (083),Preapplication Coordination (091),Application Procedures (092),Award Procedure (093),Deadlines (094),Range of Approval/Disapproval Time (095),Appeals (096),Renewals (097),Formula and Matching Requirements (101),Length and Time Phasing of Assistance (102),Reports (111),Audits (112),Records (113),Account Identification (121),Obligations (122),Range and Average of Financial Assistance (123),Program Accomplishments (130),"Regulations, Guidelines, and Literature (140)",Regional or Local Office (151),Headquarters Office (152),Website Address (153),Related Programs (160),Examples of Funded Projects (170),Criteria for Selecting Proposals (180),Published Date,Parent Shortname,URL,Recovery,Obligation Types
0,Agricultural Research Basic and Applied Research,10.001,(Extramural Research),"AGRICULTURAL RESEARCH SERVICE, AGRICULTURE, DE...","{""list"":[{""act"":{""description"":""Food Security ...","To make agricultural research discoveries, eva...",PROJECT GRANTS,Not Applicable,Usually nonprofit institutions of higher educa...,Usually nonprofit institutions of higher educa...,"{""isApplicable"":false}","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Letters should be submitted to...","A peer review panel considers each proposal, e...","{""flag"":""no"",""list"":[]}",Not Applicable,None,None,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""None"",""award...","[{""code"":""program"",""isSelected"":true,""descript...","{""isApplicable"":true,""description"":""As perform...","Financial records, supporting documents, stati...",12-1400-0-1-352;,"(Project Grants) FY 19$4,704,045.00; FY 20 est...","$5,000 to $50,000. Average $20,000","{""list"":[],""isApplicable"":false}",2 CFR 200,"{""flag"":""appendix"",""description"":""See the Agri...","Deborah Watson5601 Sunnyside Avenue, MS 5110, ...",http://www.ars.usda.gov,10.207 Animal Health and Disease Research; 10....,Not Applicable.,Peer review.,"Jan 01,1965",USDA,https://beta.sam.gov/fal/2cf5bced6e734d309ddad...,No,[Project Grants]
1,"Plant and Animal Disease, Pest Control, and An...",10.025,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Plant Protecti...",To protect U.S. agriculture from economically ...,PROJECT GRANTS,Not Applicable,"Foreign, State, local, and U.S. Territorial go...","Farmers, ranchers, agriculture producers, Stat...","{""description"":""Curriculum vitae for principal...","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Comply with E.O. 12372, \""Inte...",Applications are approved by the Administrator...,"{""flag"":""contact"",""list"":[]}",From 60 to 120 days.,None,Based on program needs and availability of ann...,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""Up to 1 year...","[{""code"":""program"",""isSelected"":false,""descrip...","{""isApplicable"":true,""description"":""N/A""}",Instruction provided in the Notice of Award. ...,12-9971-0-7-352;12-1600-0-1-352;,"(Salaries and Expenses) FY 19$288,228,684.00; ...",NaN,"{""list"":[{""fiscalYear"":2016,""description"":""Sel...","Uniform Administrative Requirements, Cost Prin...","{""flag"":""appendix"",""description"":""Consult Appe...","Eileen M. Berke,4700 River Road, Unit 55, Rive...",http://www.aphis.usda.gov/,12.100 Aquatic Plant Control; 15.611 Wildlife ...,Not Applicable.,Not Applicable.,"Jan 01,1972",USDA,https://beta.sam.gov/fal/226bb32f9354425f85946...,No,[Salaries and Expenses]
2,Wildlife Services,10.028,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Animal Damage ...",To reduce damage caused by mammals and birds a.

In [10]:
data.to_csv('AssistanceListingInProgress.csv')

from google.colab import files
files.download('AssistanceListingInProgress.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Checking Point** ----- Done with obligation sepration 
<br>
Next working on seprating the $amount for each obligation type

In [12]:
obli = obligations.str.split(';', expand=True)